In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# read csv into df
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from constants import *
from glob import glob
from datetime import datetime
import os


df = pd.DataFrame(columns=['datum', "location_id", "pretocnost"])

location_dict = {
    "kum": 0,
    "lovrenška_jezera": 1,
    "osp": 2,
    "storžič": 3,
    "triglavski_narodni_park": 4,
    "vršič": 5,
}

for path in glob(DATA_GLOB_PLANINSKE_MAKRO_ALL):
    print(path)

    # get file name without extension and path
    file_name = os.path.basename(path).split('.')[0].lower().replace(" ", "_")
    if file_name == "zdruzeno":
        continue
    
    # file_name = path.split('/')[-1].split('.')[0].replace(" ", "_").lower()
    assert file_name in location_dict.keys(), f"file_name = {file_name}"
    print(file_name)
    location_id = location_dict[file_name]

    df_planine = pd.read_csv(path, sep=',', encoding='UTF-8')
    assert df_planine.columns[0] == 'datum', f"columns[0] = {df_planine.columns[0]}"
    assert df_planine.columns[1] == 'vhodi', f"columns[1] = {df_planine.columns[1]}"
    assert df_planine.columns[2] == 'izhodi', f"columns[2] = {df_planine.columns[2]}"
    
    # sum vhodi and izhodi
    df_planine['pretocnost'] = df_planine['vhodi'] + df_planine['izhodi']
    # drop "vhodi" and "izhodi" columns
    df_planine = df_planine.drop(columns=['vhodi', 'izhodi'])

    # add "location_id" column
    df_planine['location_id'] = location_id

    df = pd.concat([df, df_planine], ignore_index=True)

    # sum vhodi and izhodi based on "datum_dan"
    # df_planine = df_planine.groupby('datum').sum()
    print(df_planine.head())

    
    # break

df = df.sort_values(by=['datum', 'location_id'])
print(df.head())

df.to_csv(OUT_PLANINSTVO, index=False)



In [18]:
# https://podatki.gov.si/dataset/seznam-praznikov-in-dela-prostih-dni-v-republiki-sloveniji

df_prazniki = pd.read_csv(DATA_PRAZNIKI, sep=';', encoding='UTF-8')
df_prazniki = df_prazniki[['DATUM', 'IME_PRAZNIKA', 'DELA_PROST_DAN']]
# DATUMM to datetime
df_prazniki['DATUM'] = pd.to_datetime(df_prazniki['DATUM'], format='%d.%m.%Y')
# keep only 2022 to 2024
df_prazniki = df_prazniki[(df_prazniki['DATUM'] >= '2022-01-01') & (df_prazniki['DATUM'] <= '2024-12-31')]
# ne and da to 0 and 1 for DELA_PROST_DAN
df_prazniki['DELA_PROST_DAN'] = df_prazniki['DELA_PROST_DAN'].str.strip()
print(df_prazniki['DELA_PROST_DAN'].unique())
assert len(df_prazniki['DELA_PROST_DAN'].unique()) == 2, f"unique = {df_prazniki['DELA_PROST_DAN'].unique()}"
df_prazniki['DELA_PROST_DAN'] = df_prazniki['DELA_PROST_DAN'].map({'ne': False, 'da': True})
# sort by date
df_prazniki = df_prazniki.sort_values(by=['DATUM'])
df_prazniki

df_prazniki.to_csv(OUT_PRAZNIKI, index=False)

['da' 'ne']


,DATUM,IME_PRAZNIKA,DELA_PROST_DAN
393,2022-01-01,novo leto,True
394,2022-01-02,novo leto,True
395,2022-02-08,"Prešernov dan, slovenski kulturni praznik",True
396,2022-04-17,velika noč,True
397,2022-04-18,velikonočni ponedeljek,True
398,2022-04-27,dan boja proti okupatorju,True
399,2022-05-01,praznik dela,True
400,2022-05-02,praznik dela,True
401,2022-06-05,binkoštna nedelja,True
402,2022-06-08,dan Primoža Trubarja,False


In [ ]:
import datetime

# convert datum to day of the week number with Monday=0, Sunday=6
df['datum'] = pd.to_datetime(df['datum'])
df['day_of_week'] = df['datum'].dt.dayofweek

# 

print(df.head())
